In [ ]:
import json
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from collections import defaultdict

# Model

In [ ]:
file1_path = 'my_spotify_data/Spotify Account Data/StreamingHistory_music_0.json'
# file2_path = 'my_spotify_data/Spotify Account Data/StreamingHistory_music_1.json'
# file3_path = 'my_spotify_data/Spotify Account Data/StreamingHistory_music_2.json'
# file4_path = 'my_spotify_data/Spotify Account Data/StreamingHistory_music_3.json'
# file5_path = 'my_spotify_data/Spotify Account Data/StreamingHistory_music_4.json'

def load_json_file(file_path): 
    encodings = ['utf-8', 'utf-8-sig', 'latin-1', 'iso-8859-1']
    
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as file:
                return json.load(file)
        except UnicodeDecodeError:
            continue
    
    raise ValueError(f"Unable to decode the file {file_path} with any of the attempted encodings.")
    
def combine_music_data(file1_path):
    data1 = load_json_file(file1_path)
    # data2 = load_json_file(file2_path)
    # data3 = load_json_file(file3_path)
    # data4 = load_json_file(file4_path)
    # data5 = load_json_file(file5_path)
    
    # Combine the data
    combined_data = data1
    
    # Convert to DataFrame
    df = pd.DataFrame(combined_data)
    
    # Convert 'endTime' to datetime and 'msPlayed' to timedelta
    df['endTime'] = pd.to_datetime(df['endTime'])
    df['duration'] = pd.to_timedelta(df['msPlayed'], unit='ms')
    
    # Drop the 'msPlayed' column as we now have 'duration'
    df = df.drop('msPlayed', axis=1)
    
    # Convert 'duration' to HH:MM:SS format
    df['duration'] = df['duration'].apply(lambda x: f"{int(x.total_seconds() // 3600):02d}:{int(x.total_seconds() % 3600 // 60):02d}:{int(x.total_seconds() % 60):02d}")
    
    # Sort by endTime
    df = df.sort_values('endTime')
    
    # Reset index
    df = df.reset_index(drop=True)
    
    return df

In [ ]:
result_df = combine_music_data(file1_path)
result_df.to_csv('datasets/combined_music_data.csv', index=False, encoding='utf-8-sig')

# Analysis

In [ ]:
df = pd.read_csv('datasets/combined_music_data.csv', parse_dates=['endTime'])

df.tail(10)

# Time Spent Over Time

In [ ]:
df_copy = df.copy()
def duration_to_hour(duration):
    h, m, s = map(int, duration.split(':'))
    return h + m / 60 + s / 3600

df_copy['duration_hours'] = df_copy['duration'].apply(duration_to_hour)
df_copy['cumulative_hours'] = df_copy['duration_hours'].cumsum()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df_copy['endTime'], df_copy['cumulative_hours'], marker='o')
plt.title("Cumulative Listening Time")
plt.xlabel("Date")
plt.ylabel("Time (hours)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Print total listening time
total_hours = df_copy['duration_hours'].sum()
print(f"Total listening time: {total_hours:.2f} hours")

### Usage by Months

In [ ]:
df_copy['month_year'] = df_copy['endTime'].dt.to_period('M')\
    
monthly_usage = df_copy.groupby('month_year')['duration_hours'].sum().reset_index()

monthly_usage = monthly_usage.sort_values('month_year')

In [ ]:
# Create the histogram
plt.figure(figsize=(12, 6))
plt.bar(monthly_usage['month_year'].astype(str), monthly_usage['duration_hours'])

plt.title("Monthly Listening Time")
plt.xlabel("Month")
plt.ylabel("Listening Time (hours)")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Add value labels on top of each bar
for i, v in enumerate(monthly_usage['duration_hours']):
    plt.text(i, v, f'{v:.2f}', ha='center', va='bottom')

plt.show()

In [ ]:
# Print average monthly listening time
avg_monthly_hours = monthly_usage['duration_hours'].mean()
print(f"Average monthly listening time: {avg_monthly_hours:.2f} hours")

# Track Counts Over Time

In [ ]:
track_counts = df['trackName'].value_counts()

top_10_tracks = track_counts.head(10)
# Create a bar chart
plt.figure(figsize=(10, 6))
top_10_tracks.plot(kind='barh')
plt.title('Number of Tracks per Artist')
plt.xlabel('Number of Tracks')
plt.ylabel('Artist')
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
print(top_10_tracks)

# Top Tracks Total Listening Time

In [ ]:
track_usage = df_copy.groupby('trackName')['duration_hours'].sum().reset_index()
top_10_tracks = track_usage.sort_values('duration_hours', ascending=False).head(10)

# Create the bar chart
plt.figure(figsize=(10, 6))
bars = plt.bar(top_10_tracks['trackName'], top_10_tracks['duration_hours'])

plt.title("Top 10 Tracks by Total Listening Time")
plt.xlabel("Track Name")
plt.ylabel("Total Listening Time (hours)")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Add value labels on top of each bar
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.2f}',
             ha='center', va='bottom')

# Adjust layout to prevent cutting off labels
plt.subplots_adjust(bottom=0.2)

plt.show()

In [ ]:
# Print total listening time for top 10 tracks
total_top_10 = top_10_tracks['duration_hours'].sum()
print(f"Total listening time for top 10 tracks: {total_top_10:.2f} hours")

# Print percentage of total listening time
total_all = df_copy['duration_hours'].sum()
percentage = (total_top_10 / total_all) * 100
print(f"This represents {percentage:.2f}% of your total listening time")

# Specific Track Listening Time

In [ ]:
# Function to get listening time for a specific track
def get_track_listening_time(track_name):
    track_time = track_usage[track_usage['trackName'] == track_name]['duration_hours'].values
    if len(track_time) > 0:
        return f"Total listening time for '{track_name}': {track_time[0]:.2f} hours"
    else:
        return f"Track '{track_name}' not found in the dataset"

# Example usage:
print(get_track_listening_time("Like I Do"))  # Replace with any track name you want to check
print(get_track_listening_time("Your Existence"))    # Another example